Introducing Idefics2: A Powerful 8B Vision-Language Model for the community

In [ ]:
%pip install -U transformers \
datasets==2.14.4 \
diffusers==0.20.0 \
accelerate==0.21.0 \
torch==2.0.1 \
torchvision==0.15.2 \
sentencepiece==0.1.99

Load the model

In [ ]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "HuggingFaceM4/idefics-9b-instruct"

model = IdeficsForVisionText2Text.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)


processor = AutoProcessor.from_pretrained(model_name)

# Generation args
exit_condition = processor.tokenizer("<end_of_utterance>", add_special_tokens=False).input_ids
bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

Load the image

In [ ]:
from PIL import Image
img = Image.open("dog.jpg")
img

Provide the Prompt and generate the response

In [ ]:
prompt = [
   "User:",
   img,
   "Describe this image."
   "Assistant:",

]

inputs = processor(prompt, return_tensors="pt").to("cuda")
generated_ids = model.generate(**inputs, eos_token_id = exit_condition, max_new_tokens=100, bad_words_ids=bad_words_ids)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)